In [1]:
from requests import get
from bs4 import BeautifulSoup as bs
from warnings import warn
from time import sleep
from random import randint
import json
import numpy as np
import multiprocessing

## Connecting to MongoDb

In [2]:
import pymongo
client=pymongo.MongoClient("mongodb://localhost:27017") 
movies_db_new=client['movies_db_new']
movies_coll=movies_db_new['movies_new_3']


In [75]:
genre_list=['action','adventure','animation','biography','comedy','crime',
        'drama','family','fantasy','film-noir','history',
           'horror','music','musical','mystery','sci-fi','romance',
           'sport','thriller','war','western']

In [56]:

headers = {'Accept-Language': 'en-US,en;q=0.8'}

## Extracting movie info from IMDB website

In [80]:


for genre in genre_list:
    pages = np.arange(1, 9952, 50)
    for page in pages:
        
        movie_list=[]
        
        response = get("https://www.imdb.com/search/title/?title_type=feature&genres=western&start="+str(page)+"&explore=genres&ref_=adv_nxt",headers=headers)
        
        
        
        if response.status_code != 200:
            warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
        page_html = bs(response.text, 'html.parser')
        
        movie_container = page_html.find_all('div', class_ = 'lister-item mode-advanced')
        
        for container in movie_container:
            
            dic=dict()
            
            
                
                
                
            dic['title'] = container.h3.a.text
                
            dic['id'] = container.h3.a['href'][7:-1]
                
            if container.h3.find('span', class_ = 'lister-item-year text-muted unbold') is not None:
                    
                dic['year'] = container.h3.find('span', class_ = 'lister-item-year text-muted unbold').text[1:-1]
                    
            else:
                    
                dic['year'] = None
                    
            if container.p.find('span', class_ = 'runtime') is not None:
                    
                dic['runtime'] = container.p.find('span', class_ = 'runtime').text
                    
            else:
                    
                dic['runtime'] = None
                    
            if container.p.find('span', class_ = 'genre') is not None:
                    
                genre = container.p.find('span', class_ = 'genre').text.replace('\n','').rstrip().split(',')
                genre = [g.strip() for g in genre]
                    
                dic['genre'] = genre
                    
            else:
                    
                dic['genre'] = None
                    
            if container.find('p', class_ ='') is not None:
                    
                cast_html = container.find('p', class_ ='').find_all('a')[1:]
                cast=[]
                for i in cast_html:
                    cast.append(i.text.strip())
                        
                    
                dic['cast'] = cast
                    
            else:
                    
                dic['cast'] = None
                    
                
                
            movie_list.append(dic)
            
        if movie_list:
                
            movies_coll.insert_many(movie_list)
            
        else:
            break
        
    break
                    


## Extracting movie keywords from TMDB API

In [4]:

    
for movie_info in list(movies_coll.find()):
    
    tmdb_response = get("https://api.themoviedb.org/3/movie/"+movie_info['tmdb_id']+"/keywords?api_key=ab8914eb7b88355eadb1c85b341016e3")
    
    if tmdb_response.status_code == 200:
                    
        tmdb_json = json.loads(tmdb_response.content)
        
        if tmdb_json['keywords']:
            
            keywords=[]
            
            for keyword in tmdb_json['keywords']:
                keywords.append(keyword['name'])
                
                  
            movies_coll.update_one(movie_info, { "$set": { "keywords": keywords } })
            
        else:
            
            movies_coll.delete_one(movie_info)
    



## Extracting language, overview and image of movies from TMDB API

In [1]:
tmdb_response = get("https://api.themoviedb.org/3/find/"+dic['id']+"?api_key=ab8914eb7b88355eadb1c85b341016e3&language=en-US&external_source=imdb_id")
                
                if tmdb_response.status_code == 200:
                    
                    tmdb_json = json.loads(tmdb_response.content)
                    
                    if tmdb_json['movie_results']:
                        
                        dic['lang'] = tmdb_json['movie_results'][0]['original_language']
                    
                        dic['overview'] = tmdb_json['movie_results'][0]['overview']
                    
                else:
                    
                    dic['lang'] = None
                    
                    dic['overview'] = None
                    
                image = get("https://www.imdb.com/title/"+dic['id']+"/")
                
                if image.status_code == 200:
                    
                    image=bs(image.content,'html.parser')
                    
                    dic['image'] = image.find('meta',attrs={'property':"og:image"})['content']
                    
                else:
                    
                    dic['image'] = None
                    

IndentationError: unexpected indent (4134970124.py, line 3)